In [7]:
import pandas as pd
import train

df = pd.read_csv('cleaneddata.csv')
# df

/var/folders/_4/yzd4h3k95051tj7kpg425rk80000gn/T/ipykernel_82126/3936584759.py:4: DtypeWarning: Columns (11,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('cleaneddata.csv')


In [3]:
print("Column 11 name:", df.columns[11])
print("Column 24 name:", df.columns[24])

Column 11 name: DateOfCancel
Column 24 name: cancel_timing


In [39]:
pd.set_option('display.max_columns', None)

print(df.head())

          DateOfOrder   OrderId                     TransactionId  \
0 2019-12-02 20:39:46  11161800  5fff5813ae674809a06e3b1fe5528fcf   
1 2019-12-02 20:39:46  11161801  7c4f0e97356e46669abefe8dc5fb0ff0   
2 2019-12-03 22:29:24  11166739  ee1d1a4a7458414a8bb420111ba0fa8c   
3 2019-12-04 13:27:22  11169532  86497dc4e7434b838a335c439325fb65   
4 2019-12-04 13:27:22  11169533  6054345ad0b14bd297dbdc6372ff1488   

  DateOfService  OrderQty          MenuName  MenuPrice  MenuSubsidy  \
0    2020-01-02         1  Mittagessen (BS)       3.05          0.0   
1    2020-01-03         1  Mittagessen (BS)       3.05          0.0   
2    2020-01-02         1  Mittagessen (BS)       3.05          0.0   
3    2020-01-02         1  Mittagessen (BS)       3.05          0.0   
4    2020-01-03         1  Mittagessen (BS)       3.05          0.0   

       BookingNr       GroupName  CanceledQty DateOfCancel Site SchoolID  \
0  474-136052114  Bad Sassendorf            0          NaN   LP   SCH003   
1  474

In [ ]:
holiday= df[df['is_holiday']== 1]
holiday

In [38]:
# checking if there is still cases remaing wich have Dateofservice before dateoforder

df['DateOfOrder'] = pd.to_datetime(df['DateOfOrder'])
df['DateOfService'] = pd.to_datetime(df['DateOfService'])

filtered_df = df[df['DateOfService'] < df['DateOfOrder'].dt.date]
